<a href="https://colab.research.google.com/github/wanadzhar913/aitinkerers-hackathon-supa-team-werecooked/blob/master/notebooks-benchmarking-exercises/demo_aitinkerers_24102024.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install flash_attn accelerate bitsandbytes -U -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 82.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 330.9/330.9 kB 30.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.4/122.4 MB 18.8 MB/s eta 0:00:00


In [2]:
import re
import json
from glob import glob
from typing import Dict
from tqdm.notebook import tqdm

import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, \
                         BitsAndBytesConfig, pipeline

In [3]:
TORCH_DTYPE = 'bfloat16'

nf4_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type='nf4',
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=getattr(torch, TORCH_DTYPE)
)

In [5]:
tokenizer = AutoTokenizer.from_pretrained('wanadzhar913/malaysian-mistral-llmasajudge-v3')
model = AutoModelForCausalLM.from_pretrained(
    'wanadzhar913/malaysian-mistral-llmasajudge-v3',
    use_flash_attention_2 = True,
    quantization_config = nf4_config
)

tokenizer_config.json:   0%|          | 0.00/1.64k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.79M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/624 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/686 [00:00<?, ?B/s]

`low_cpu_mem_usage` was None, now set to True since model is quantized.


...

model-00002-of-00003.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

The model was loaded with use_flash_attention_2=True, which is deprecated and may be removed in a future release. Please use `attn_implementation="flash_attention_2"` instead.


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

In [6]:
pipe = pipeline(
    "text-generation",
    tokenizer = tokenizer,
    model=model,
)

In [15]:
prompt = """Anda adalah pakar dalam mengesan ketidakkonsistenan fakta dan halusinasi. Anda akan diberi satu dokumen dan satu soalan. Baca
dokumen dan soalan/kenyataan yang diberikan dengan teliti dan kenal pasti Ketidakkonsistenan Fakta (iaitu mana-mana soalan/kenyataan yang
tidak disokong atau bercanggah dengan maklumat dalam dokumen).

### Anda perlu memilih antara dua pilihan berikut:
- Tidak Konsisten dengan Fakta: Jika mana-mana soalan/kenyataan tidak disokong, terjawab atau bercanggah dengan dokumen, labelkannya sebagai 0.
- Konsisten dengan Fakta: Jika semua soalan/kenyataan disokong/terjawab oleh dokumen, labelkannya sebagai 1.

### Sebagai contoh:
Dokumen: "Gajah adalah mamalia besar yang biasanya ditemui di Afrika dan Asia. Mereka hidup dalam kumpulan yang dikenali sebagai kawanan dan terkenal kerana mempunyai ingatan yang baik."

Soalan/Kenyataan: "Gajah adalah mamalia besar yang biasanya ditemui di Eropah."
Jawapan: {{'consistency': 0}}

Soalan/Kenyataan: "Gajah adalah mamalia besar yang biasanya ditemui di Afrika dan Asia."
Jawapan: {{'consistency': 1}}

### Jawab berdasarkan dokumen dan soalan/kenyataan berikut:
Dokumen: {passage}
Soalan/Kenyataan: {question}

Sediakan penjelasan langkah demi langkah untuk pilihan konsistenan berdasarkan Dokumen dan Soalan/Kenyataan yang diberikan. Selepas itu,
kembalikan pilihan konsistenan dalam format JSON untuk pilihan yang diberikan. Sebagai contoh: {{'consistency': 1}} atau {{'consistency': 0}}"""

In [20]:
# https://www.thestar.com.my/business/business-news/2024/10/23/strong-support-for-chip-sector-under-budget-2025
passage_english = """
KUALA LUMPUR: Budget 2025 has set aside sizeable funds, both fiscal and non-fiscal, to ensure the success of the National Semiconductor Strategy (NSS), which is part of the New Industrial Master Plan 2030 (NIMP 2030), says Investment, Trade and Industry (Miti) Minister Tengku Datuk Seri Zafrul Abdul Aziz.

Among the initiatives announced in the budget, he said were the RM1bil sovereign fund for the electrical and electronics sector and high-value activities as well as training funds allocated for several universities.

Apart from that, he said there are initiatives to support mid-tier companies as well as tax incentives for companies in the industry.

“I think we are on track (to achieve the target set in NIMP 2030). You have seen exports continue to grow in these sectors as well.

“And if you look at the just-announced report card for our NIMP 2030, we should see positive growth by year-end, and growth in the manufacturing sector has contributed close to a 5% increase to our gross domestic product this year,” he said this during an interview with CNBC Asia Squawk Box yesterday.

Tengku Zafrul was commenting on the progress of the NSS and NIMP.

When asked how the new tax would help finance the bigger budget of RM421bil, he said that apart from the tax on dividends as well as the larger scope of sales and service tax, emphasis is given on cost discipline, for instance, via the merging of several agencies under Miti.

“Yes, I am quite confident that we will meet the budget estimate. We have been meeting our deficit target, for example, and I think we will hopefully achieve it (fiscal target) in 2024,” he said.

The ministry will also continue with initiatives to drive trade and investments to spur the country’s growth, added Tengku Zafrul. — Bernama
"""

In [21]:
question_english = "Zafrul will not meet the budget deficit."

In [27]:
print(
    pipe(
        prompt.format(passage=passage_english, question=question_english),
        max_new_tokens = 1024,
        return_full_text=False,
        temperature = 0.1,
        do_sample = True,
        top_p = 0.97,
        top_k = 50,
    )[0]['generated_text']
)

 [/INST]Untuk menentukan sama ada pernyataan "Zafrul will not meet the budget deficit" adalah konsisten dengan petikan yang diberikan, kita perlu menganalisis maklumat yang terdapat dalam petikan tersebut.

1. **Menganalisis Petikan**: 
   - Petikan menyatakan bahawa Menteri Tengku Zafrul Abdul Aziz yakin bahawa mereka akan memenuhi anggaran defisit.
   - Ia juga menyebut bahawa Malaysia telah berjaya memenuhi sasaran defisit pada masa lalu dan berharap untuk mencapai sasaran fiskal pada tahun 2024.

2. **Menganalisis Pernyataan**:
   - Pernyataan yang diberikan adalah "Zafrul will not meet the budget deficit". Ini menunjukkan bahawa Zafrul tidak akan mencapai sasaran defisit yang ditetapkan.

3. **Membandingkan Maklumat**:
   - Dari petikan, kita tahu bahawa Zafrul menyatakan keyakinan bahawa mereka akan memenuhi anggaran defisit. Ini bertentangan dengan pernyataan yang menyatakan bahawa Zafrul tidak akan memenuhi defisit bajet.

4. **Kesimpulan**:
   - Oleh kerana pernyataan tersebut

In [17]:
# https://www.astroawani.com/berita-malaysia/harga-runcit-ron97-ron95-dan-diesel-kekal-493134
passage_malay = """
KUALA LUMPUR: Harga runcit petrol RON97 dan RON95 di seluruh negara masing-masing kekal pada paras RM3.19 seliter dan RM2.05 seliter bagi tempoh 24 hingga 30 Okt ini.
Kementerian Kewangan dalam kenyataan hari ini memaklumkan harga runcit diesel di Semenanjung Malaysia kekal pada paras RM2.95 seliter manakala di Sabah, Sarawak dan Labuan turut tidak berubah, iaitu RM2.15 seliter bagi tempoh berkenaan.
Penetapan harga itu dibuat berdasarkan harga runcit produk petroleum secara mingguan dengan menggunakan formula Mekanisme Harga Automatik.

"Kerajaan juga akan terus memantau kesan perubahan harga minyak mentah dunia serta mengambil langkah yang bersesuaian bagi memastikan kebajikan dan kesejahteraan rakyat terus terpelihara," menurut kenyataan itu.
-- BERNAMA
"""

In [22]:
question_malay = "harga runcit diesel di Sabah tak ubah juga."

In [30]:
def call_llm(passage: str, question: str) -> str:
    print(
        pipe(
            prompt.format(passage=passage_malay, question=question_malay),
            max_new_tokens = 1024,
            return_full_text=False,
            temperature = 0.1,
            do_sample = True,
            top_p = 0.97,
            top_k = 50,
        )[0]['generated_text']
    )

 [/INST]Untuk menentukan sama ada pernyataan "harga runcit diesel di Sabah tak ubah juga" adalah konsisten dengan petikan yang diberikan, kita boleh mengikuti langkah-langkah berikut:

1. **Menganalisis Petikan**: Dalam petikan, dinyatakan bahawa harga runcit diesel di Semenanjung Malaysia adalah RM2.15 seliter, manakala di Sabah, Sarawak, dan Labuan juga kekal pada harga yang sama, iaitu RM2.15 seliter. Ini menunjukkan bahawa harga diesel di Sabah tidak berubah.

2. **Menganalisis Pernyataan**: Pernyataan yang diberikan adalah "harga runcit diesel di Sabah tak ubah juga". Ini merujuk kepada fakta bahawa harga diesel di Sabah tidak berubah.

3. **Membandingkan Maklumat**: Dari petikan, kita dapat melihat bahawa harga diesel di Sabah memang tidak berubah, yang menyokong pernyataan tersebut.

4. **Kesimpulan**: Oleh kerana petikan menyatakan bahawa harga diesel di Sabah kekal pada RM2.15 seliter, maka pernyataan "harga runcit diesel di Sabah tak ubah juga" adalah konsisten dengan makluma